## Single Server Queue Simulation

linux commit

### Import Library

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd

### Initialization Function

In [2]:
class queueSim:
    def __init__(self, nextTime, servTime, totalCust, arrTime, endServ):
        self.nextTime = nextTime
        self.servTime = servTime
        self.totCust = totalCust
        self.arrTime = arrTime
        self.endServ = endServ
        self.coIndex = 0
        
    def simulation(self):
        time = 0

        for i in range(self.totCust):
            #arriveList[i] = dt.datetime.fromtimestamp(time).strftime('%H:%M')

            time += self.nextTime[i]
            self.arrTime[i] = max(time, self.endServ[i])
            self.endServ[i] = self.arrTime[i] + self.servTime[i]
            
        self.arrTime = self.arrTime[(self.arrTime < 7.5 * 3600)]
        self.endServ = self.endServ[0:len(self.arrTime)]
        self.totCust = len(self.arrTime)
        
    def getStartTime(self):
        startTime = np.empty([self.totCust], dtype='U5')
        customer = np.empty([self.totCust, 1], dtype=int)
        
        for i in range(self.totCust):
            temp = self.arrTime[i]
            startTime[i] = dt.datetime.fromtimestamp(temp).strftime('%H:%M')
            customer[i] = i + 1
        
        activity = np.array(['Arrive'] * self.totCust).reshape([self.totCust, 1])
        
        return np.concatenate((startTime.reshape([self.totCust, 1]), customer, activity), axis=1)
    
    def getEndTime(self):
        endTime = np.empty([self.totCust], dtype='U5')
        customer = np.empty([self.totCust, 1], dtype=int)
        
        for i in range(self.totCust):
            temp = self.endServ[i]
            endTime[i] = dt.datetime.fromtimestamp(temp).strftime('%H:%M')
            customer[i] = i + 1
        
        activity = np.array(['Depart'] * self.totCust).reshape([self.totCust, 1])
        
        return np.concatenate((endTime.reshape([self.totCust, 1]), customer, activity), axis=1)
    
    def getTimeSeq(self):
        timeSeq = (np.concatenate((self.getStartTime(), self.getEndTime())))
        
        return timeSeq
    
    def getTimeStamp(self):
        return np.sort(np.concatenate((self.arrTime, self.endServ)))
    
    def getDataFrame(self):
        df = pd.DataFrame(queue1.getTimeSeq(), columns=['Time', 'Customer', 'Activity']).sort_values(by=['Time'], ignore_index=True)
        
        custInTime = np.zeros([self.totCust * 2]).astype(int)
        serverStatus = np.empty([self.totCust * 2], dtype='U9')
        temp = 0
        
        for i in range(self.totCust * 2):
            if df.loc[i, 'Activity'] == 'Arrive':
                temp += 1

            elif df.loc[i, 'Activity'] == 'Depart':
                temp -= 1

            custInTime[i] = temp

            if custInTime[i] == 0:
                serverStatus[i] = 'AVAILABLE'

            else:
                serverStatus[i] = 'BUSY'
                
        df['numberOfCustomers'] = custInTime
        df['serverStatus'] = serverStatus
        
        df.loc[-1] = ['07:00', None, 'Opening', 0, 'AVAILABLE']
        df.loc[-2] = ['14:30', None, 'Close Order', 0, 'AVAILABLE']
        df.loc[-3] = ['15:00', None, 'Closing', 0, 'BUSY']
        
        df = df.sort_values(by=['Time']).reset_index(drop=True)
        
        self.coIndex = df.Activity[df.Activity == 'Close Order'].index.to_list()[0]
        df.loc[self.coIndex, ['numberOfCustomers', 'serverStatus']] = df.loc[self.coIndex - 1, ['numberOfCustomers', 'serverStatus']]
        
        return df

### Simulation

In [5]:
targetCust = 25
nextArrTime = np.random.normal(25, 3, targetCust).astype(int) * 60
serviceTime = np.random.normal(23, 5, targetCust).astype(int) * 60
arrTime = np.zeros([targetCust]).astype(int)
endService = np.zeros([targetCust]).astype(int)

queue1 = queueSim(nextArrTime, serviceTime, targetCust, arrTime, endService)
queue1.simulation()

queue1.getDataFrame()

,Time,Customer,Activity,numberOfCustomers,serverStatus
0,07:00,None,Opening,0,AVAILABLE
1,07:29,1,Arrive,1,BUSY
2,07:49,1,Depart,0,AVAILABLE
3,07:52,2,Arrive,1,BUSY
4,08:11,2,Depart,0,AVAILABLE
5,08:14,3,Arrive,1,BUSY
6,08:40,3,Depart,0,AVAILABLE
7,08:42,4,Arrive,1,BUSY
8,08:56,4,Depart,0,AVAILABLE
9,09:11,5,Arrive,1,BUSY
